# Modelagem

Inicialmente, o conjunto de dados foi dividido em conjuntos de treinamento e teste por meio de amostragem estratificada. O conjunto de teste foi isolado antes de qualquer etapa de pré-processamento. 

Todas as etapas de transformação, balanceamento, seleção de algoritmo e ajuste de hiperparâmetros foram realizadas exclusivamente no conjunto de treinamento utilizando validação cruzada estratificada. O modelo final foi então treinado com todos os dados de treinamento e avaliado uma única vez no conjunto de teste.

### Fluxograma do Pipeline de Dados

```text
       ┌──────────────────────────────────────────┐
       │     Dataset original (desbalanceado)     │
       │            ~10.000 registros             │
       └──────────────────────────────────────────┘
                     │
                     │ (Split estratificado)
                     ▼
       ┌──────────────────────────┐      ┌──────────────────────────────────────────┐
       │     Conjunto de Teste    │      │         Conjunto de Treinamento          │
       │   (~20% | ~2.000 reg.)   │◄─────┤           (~80% | ~8.000 reg.)           │
       │  ❌ Sem pré-processamento │      └──────────────────────────────────────────┘
       │  ❌ Sem balanceamento     │                    │
       └─────────────┬────────────┘                    ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Pré-processamento    │
                     │                 │ (scaler, encoder, etc.)  │
                     │                 │   ✔ fit apenas no treino │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │      Balanceamento       │
                     │                 │ (SMOTE / Under / Weights)│
                     │                 │  ❗ apenas no treino      │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Validação Cruzada    │
                     │                 │    (Stratified K-Fold)   │
                     │                 │                          │
                     │                 │ • Comparar algoritmos    │
                     │                 │ • Ajustar hiperparâmetros│
                     │                 │ • Estimar desempenho     │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │  Modelo final escolhido  │
                     │                 │ (melhor algoritmo +      │
                     │                 │   melhores parâmetros)   │
                     │                 └──────────────────────────┘
                     │                                 │
                     │                                 ▼
                     │                 ┌──────────────────────────┐
                     │                 │     Treinamento final    │
                     │                 │      (100% do treino)    │
                     │                 │      ❌ sem CV           │
                     │                 └──────────────────────────┘
                     │                                 │
                     ▼                                 ▼
       ┌──────────────────────────────────────────────────────────┐
       │                 Avaliação final no teste                 │
       │ • Métricas reais (Precisão, AUC, curva ROC,              |
       | sensibilidade e especificidade)                          │
       │ • Validação de Generalização                             │
       │ • Executado uma única vez                                │
       └──────────────────────────────────────────────────────────┘

### Separando conjunto de teste e de treinamento

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

from utils.general import *
from utils.norm_test import *
from utils.pre_process import *
from utils.analise_estatistica  import *

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data_path = catchPath('data/data.xlsx')
data_path

'/ML/data/data.xlsx'

In [7]:
df = pd.read_excel(data_path)
df.head()

,ID_Cliente,Status_Cliente,Idade,Genero,Numero_Dependentes,Nivel_Educacao,Estado_Civil,Faixa_Renda,Categoria_Cartao,Meses_Relacionamento,...,Contatos_12m,Limite_Credito,Saldo_Rotativo,Limite_Disponivel,Variacao_Valor_Transacoes,Valor_Total_Transacoes,Quantidade_Transacoes,Variacao_Qtd_Transacoes,Taxa_Utilizacao_Credito,classe
0,768805383,Cliente Ativo,45,Masculino,3,Ensino Médio,Casado,60k a 80k,Azul,39,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,1
1,818770008,Cliente Ativo,49,Feminino,5,Graduação,Solteiro,Menos de 40k,Azul,44,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,1
2,713982108,Cliente Ativo,51,Masculino,3,Graduação,Casado,80k a 120k,Azul,36,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,1
3,769911858,Cliente Ativo,40,Feminino,4,Ensino Médio,Não Informado,Menos de 40k,Azul,34,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,1
4,709106358,Cliente Ativo,40,Masculino,3,Sem Escolaridade,Casado,60k a 80k,Azul,21,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,1


In [8]:
df.shape

(10127, 22)

### Tabela Final de Decisão de Features

| Variável | Evidência Estatística | Correlação | Decisão |
|--|--|--|--|
| Quantidade_Transacoes | Forte | Alta com Valor_Total | Manter |
| Taxa_Utilizacao_Credito | Moderada | Baixa | Manter |
| Meses_Inativos_12m | Moderada | Baixa | Manter |
| Variacao_Qtd_Transacoes | Significativa | Baixa | Manter |
| Variacao_Valor_Transacoes | Significativa | Baixa | Manter |
| Valor_Total_Transacoes | Moderada | Alta | Substituir |
| Limite_Credito | Fraca | Moderada | Opcional |
| Idade | Não significativa | — | Remover |


In [9]:
df.columns

Index(['ID_Cliente', 'Status_Cliente', 'Idade', 'Genero', 'Numero_Dependentes',
       'Nivel_Educacao', 'Estado_Civil', 'Faixa_Renda', 'Categoria_Cartao',
       'Meses_Relacionamento', 'Total_Produtos', 'Meses_Inativos_12m',
       'Contatos_12m', 'Limite_Credito', 'Saldo_Rotativo', 'Limite_Disponivel',
       'Variacao_Valor_Transacoes', 'Valor_Total_Transacoes',
       'Quantidade_Transacoes', 'Variacao_Qtd_Transacoes',
       'Taxa_Utilizacao_Credito', 'classe'],
      dtype='object')

In [10]:
features = [
    'Quantidade_Transacoes',
    'Taxa_Utilizacao_Credito',
    'Meses_Inativos_12m',
    'Variacao_Qtd_Transacoes',
    'Variacao_Valor_Transacoes'    
]

In [12]:
X = df[features].copy()
X.shape

(10127, 5)

In [13]:
y = df['classe'].copy()
y.shape

(10127,)

In [14]:
# split ANTES do balanceamento
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=2
)

In [15]:
X_test.shape

(2026, 5)

In [3]:
# determinando o nivel de confiança para a amostra de teste com um erro de 2%
df_validacao = calcular_confianca_amostral(N=10127, n=2026, e=0.02)
print(df_validacao)

                Parâmetro          Valor
0     População Total (N)         10.127
1  Tamanho da Amostra (n)  2.026 (20.0%)
2      Margem de Erro (e)           2.0%
3      Escore-Z calculado           2.01
4      Nível de Confiança          95.6%


A amostragem para construir o conjunto de dados para realizar os testes foi validado estatisticamente demonstrando representatividade. Para uma população total de $N = 10.127$, a amostra de teste selecionada de $n = 2.026$ (20% do dataset) resulta em um nível de confiança de aproximadamente 95,5%, com uma margem de erro estrita de apenas 2%.